In [5]:
import rasterio
import numpy as np
from rasterio.enums import Resampling

In [6]:
#Pre widfire images     12 August 2020
NIR_pre = rasterio.open('T10SFF_20200812T184921_B08_10m.jp2')
SWIR_pre = rasterio.open('T10SFF_20200812T184921_B12_20m.jp2')

#Post wildfire images   11 October 2020
NIR_post = rasterio.open('T10SFF_20201011T185321_B08_10m.jp2')
SWIR_post = rasterio.open('T10SFF_20201011T185321_B12_20m.jp2')

In [7]:
#Resample to apply arithmetic band operations
def resample(NIR_band, SWIR_band, output_name ):
    resample_factor = int(NIR_band.width / SWIR_band.width)
    resampled_SWIR  = SWIR_band.read(out_shape=(SWIR_band.count,SWIR_band.height * resample_factor,
                                          int(SWIR_band.width * resample_factor)),
                                          resampling=Resampling.bilinear)
    resampled_meta = SWIR_band.meta.copy()
    resampled_meta.update({"driver": "GTiff",
                        'height': resampled_SWIR.shape[1],
                        'width': resampled_SWIR.shape[2],
                        'transform': NIR_band.transform,
                        'crs': SWIR_band.crs})
    with rasterio.open(output_name, 'w', **resampled_meta) as clip:
            clip.write(resampled_SWIR)
            clip.close()

In [8]:
#Call resample function with a proper parameters
resample(NIR_pre, SWIR_pre, 'SWIR_pre_res.tif')
resample(NIR_post, SWIR_post, 'SWIR_post_res.tif')